In [7]:
from utils.preprocessor import *
import contractions
import spacy
from utils.model import Encoder, DecoderAttn
from utils.train import training
from utils.evaluate import evaluateRandom, translate_sentence, calcBLEU, calcMetrics, calcMetrics
import torch
import torch.nn as nn
from torch import optim

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
root_path = "data"
train_data, test_data, val_data = get_dataset("train.txt", root_path=root_path),\
                            get_dataset("test.txt", root_path=root_path),\
                            get_dataset("validation.txt", root_path)
print(train_data, val_data, test_data)

Dataset({
    features: ['en', 'vi'],
    num_rows: 29000
}) Dataset({
    features: ['en', 'vi'],
    num_rows: 1014
}) Dataset({
    features: ['en', 'vi'],
    num_rows: 1000
})


#**Tiền xử lý**#

In [10]:
lst_check = "‘’“”…—"
for char in lst_check:
    for x in train_data['vi']:
        a = x.count(char)
        if a > 0 and a % 2 == 0:
            print(x)
    for x in train_data['en']:
        a = x.count(char)
        if a > 0 and a % 2 == 0:
            print(x)

In [11]:
def update_contractions(text):
    # slang = False !
    return contractions.fix(text, slang=False)
def remove_char_quotes(text):
    rs = text
    cnt1, cnt2 = text.count('"'), text.count("'")
    if cnt1 > 0 and cnt1 % 2 == 0:
        rs = text.replace('"','')
    if cnt2 > 0 and cnt2 % 2 == 0:
        rs = text.replace("'", '')
    return rs
def processing(texts):
    texts['en'], texts['vi'] = [remove_char_quotes(update_contractions(t)).lower().strip() for t in [texts['en'], texts['vi']]]
    return texts
train_data = train_data.map(processing)
test_data = test_data.map(processing)
val_data = val_data.map(processing)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

In [12]:
train_data[0]

{'en': 'two young, white males are outside near many bushes.',
 'vi': 'hai thanh niên da trắng ở bên ngoài gần nhiều bụi cây.'}

In [13]:
sos_token = 0
eos_token = 1
unk_token = 2
max_length = 25

In [14]:
en_nlp = spacy.blank("en")
vi_nlp = spacy.blank('vi')

In [15]:
def filter_data(texts):
    return len(en_nlp.tokenizer(texts['en'])) <= max_length and len(vi_nlp.tokenizer(texts['vi'])) <= max_length
train_data = train_data.filter(filter_data)
val_data = val_data.filter(filter_data)
test_data = test_data.filter(filter_data)

Filter:   0%|          | 0/29000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1014 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
train_data, test_data, val_data

(Dataset({
     features: ['en', 'vi'],
     num_rows: 28181
 }),
 Dataset({
     features: ['en', 'vi'],
     num_rows: 963
 }),
 Dataset({
     features: ['en', 'vi'],
     num_rows: 984
 }))

In [17]:
en_vocab = Vocabulary('en', train_data['en'], en_nlp)
vi_vocab = Vocabulary('vi', train_data['vi'], vi_nlp)

In [18]:
en_vocab.get_index("meomeo")

2

In [19]:
indexesFromSentence(en_vocab, "today meomeo", en_nlp)

[8572, 2]

#**Tham số mô hình**#

In [20]:
hidden_size = 256
batch_size = 128
num_layers = 1
dropout = 0.2
learning_rate = 0.001
num_epochs = 369

In [ ]:
train_loader = get_dataloader(train_data, en_vocab, vi_vocab, \
                                    en_nlp, vi_nlp, device, batch_size,\
                                    max_length, 0, 1)

In [27]:
encoder = Encoder(en_vocab.n_words, hidden_size, num_layers, dropout).to(device)
decoder = DecoderAttn(hidden_size, vi_vocab.n_words, device, max_length, 0, 1,2, num_layers, dropout).to(device)

encoder_opt = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_opt = optim.Adam(decoder.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss(ignore_index = 1) # ignore eos_token

In [ ]:
his_loss = training(train_loader, encoder, decoder, encoder_opt, decoder_opt, criterion, num_epochs, 5)

#**Lưu mô hình**#

In [21]:
enc_path = "encoder.pth"
dec_path = "decoder.pth"

In [ ]:
torch.save(encoder.state_dict(), enc_path)
torch.save(decoder.state_dict(), dec_path)

In [22]:
enc = Encoder(en_vocab.n_words, hidden_size, num_layers, dropout).to(device)
dec = DecoderAttn(hidden_size, vi_vocab.n_words, device, max_length, 0, 1, 2, num_layers, dropout).to(device)

enc.load_state_dict(torch.load(enc_path, weights_only=True))
dec.load_state_dict(torch.load(dec_path, weights_only=True))

enc.eval()
dec.eval()

DecoderAttn(
  (embedding): Embedding(6067, 256)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=256, out_features=256, bias=True)
    (Ua): Linear(in_features=256, out_features=256, bias=True)
    (Va): Linear(in_features=256, out_features=1, bias=True)
  )
  (lstm): LSTM(512, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=6067, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

#**Samples**#

In [23]:
evaluateRandom(enc, dec, train_data, en_vocab, vi_vocab, en_nlp, vi_nlp, device, 10)

English:	 a kite surfer and a kite flier prepare to launch their respective kites on a grassy slope.
Vi_true:	 một người lướt ván diều và một người thả diều chuẩn bị thả diều của mình trên một sườn dốc cỏ.
Vi_pred:	 một người lướt ván diều và một người thả diều chuẩn bị thả diều của mình trên một sườn dốc cỏ . a . phía trên một
------------
English:	 a group of africans running away from something.
Vi_true:	 một nhóm người châu phi đang chạy trốn khỏi một thứ gì đó.
Vi_pred:	 một nhóm người châu phi đang chạy trốn khỏi một thứ gì đó . . mình . .
------------
English:	 one man is holding a beer and the other is taking a bite of food.
Vi_true:	 một người đàn ông đang cầm một cốc bia và người kia đang cắn một miếng thức ăn.
Vi_pred:	 một người đàn ông đang cầm một cốc bia và người kia đang cắn một miếng thức ăn . . . ăn . .
------------
English:	 a smiling worker, in a cowboy hat, doing masonry.
Vi_true:	 một công nhân mỉm cười, đội mũ cao bồi, đang làm việc xây dựng.
Vi_pred:	 một công n

In [24]:
evaluateRandom(enc, dec, val_data, en_vocab, vi_vocab, en_nlp, vi_nlp, device, 10)

English:	 two people eat hamburgers on lawn chairs while a third drinks a can of soda.
Vi_true:	 hai người ăn bánh mì kẹp thịt trên ghế xếp trong khi người thứ ba uống một lon soda.
Vi_pred:	 hai người ăn tối trên những chiếc ghế trên những chiếc ghế trong khi làm đồ uống một lon soda . . .
------------
English:	 two brunette women talking to a classroom full of blond children.
Vi_true:	 hai người phụ nữ tóc nâu đang nói chuyện với một lớp học toàn trẻ em tóc vàng.
Vi_pred:	 hai phụ nữ tóc nâu đang nói chuyện với một lớp học đầy những đứa trẻ sơ sinh . . noble . a . noble . a . noble
------------
English:	 a guy grinds a windowsill near an old windmill
Vi_true:	 một anh chàng mài bệ cửa sổ gần một cối xay gió cũ
Vi_pred:	 một anh chàng đang trượt dán băng đồng gần một chiếc ô tô cũ hỏng . c . cũ . noble . cũ .
------------
English:	 2 females, 1 from germany and 1 from china, compete in a wrestling match on a mat.
Vi_true:	 2 cô gái, 1 người đức và 1 người trung quốc, đang tham gia một

In [25]:
inp_texts = ['people wearing the dirty jeans and riding a bicycle together']
translate_sentence(enc, dec, inp_texts, en_vocab, vi_vocab, en_nlp, device)

['mặc quần jean , áo sơ mi và cùng nhau đi xe đạp cùng nhau . ` .']

In [26]:
evaluateRandom(enc, dec, val_data, en_vocab, vi_vocab, en_nlp, vi_nlp, device, 5)

English:	 two men, one man selling fruit the other inspecting the fruit and conversing with the seller.
Vi_true:	 hai người đàn ông, một người bán trái cây, người kia đang kiểm tra trái cây và trò chuyện với người bán.
Vi_pred:	 hai người đàn ông , một người đàn ông bán trái cây khác nhau mà người bán trái cây và người bán người bán hàng . . người .
------------
English:	 man in a suit smoking a cigar and reading a paper walking on the street.
Vi_true:	 một người đàn ông mặc vest đang hút xì gà và đọc báo đang đi trên phố.
Vi_pred:	 người đàn ông mặc vest đang hút xì gà và đọc báo đi bộ trên phố . com .
------------
English:	 a boy rides a swing.
Vi_true:	 một cậu bé đang đu đưa.
Vi_pred:	 xích đu . a . a . a . a . a . a . a . a . a . a . a . a
------------
English:	 a closeup of a child's face eating a blue, heart shaped lollipop.
Vi_true:	 cận cảnh khuôn mặt của một đứa trẻ đang ăn một cây kẹo mút hình trái tim màu xanh.
Vi_pred:	 cận cảnh một đứa trẻ đang ăn một chiếc tất màu xanh ,

In [27]:
calcBLEU(enc, dec, test_data,  en_vocab, vi_vocab, en_nlp, vi_nlp, device)

0.35866784119535605

In [29]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [41]:
avg_rouge = calcMetrics(enc, dec, test_data, en_vocab, vi_vocab, en_nlp, vi_nlp, device)
print(f"Rouge: {avg_rouge[1]}")

Rouge: {'rouge1': 0.41036867034226426, 'rouge2': 0.19703308825517854, 'rougeL': 0.37877149095348234}
